In [1]:
import numpy as np
from limit_states.g11d_electric import g11d_electric
from methods.sghmc import *
from utils.data import get_dataloader
from active_training.active_train import ActiveTrain
from tqdm import tqdm

import torch.utils.data as data

In [2]:
lstate = g11d_electric()
act_train = ActiveTrain()
pf, beta, _,_, y_mc_test = lstate.monte_carlo_estimate(1e6)
y_max = np.max(y_mc_test)   #to normalise the output for training
print('ref PF:', pf, 'B:',beta)

engine humming
ref PF: 0.000207 B: 3.53099472767886


In [3]:
#Passive training
x_norm, x_scaled, y_scaled = lstate.get_doe_points(n_samples=100, method='lhs')

x_train = torch.tensor(x_norm, dtype=torch.float32).view(-1,11)
y_train = torch.tensor(y_scaled/y_max, dtype=torch.float32).view(-1,1)  #normalised output

In [4]:
# Active training
use_cuda = torch.cuda.is_available()
# for i in tqdm(range(epoch, nb_epochs)):

n_active_ep = 20
active_points = 5
mcs_samples = int(1e5)
it_count = 0

burn_in = 10   #How many epochs to burn in for?. Default: 20.
sim_steps = 2   #How many epochs pass between saving samples. Default: 2.
N_saves=10
resample_its = 50
resample_prior_its = 15
re_burn = 1e8

n_passive_ep = 50

for act_ep in range(n_active_ep):
    
    print('Samples: ', x_train.shape[0])
    
    train_loader, _ = get_dataloader(x_train, y_train, lstate.input_dim, lstate.output_dim, train_test_split=1.0, batch_size=32)

    net = BNN_SGHMC(N_train=len(x_train), input_dim=11, width=20, depth=2, output_dim=1, 
              lr=1e-2, cuda=use_cuda, grad_std_mul=10)

    cost_train = np.zeros(n_passive_ep)
    for pass_ep in tqdm(range(0, n_passive_ep)):
        net.set_mode_train(True)
        nb_samples = 0

        # passive training with current dataset
        for x, y in train_loader:
            
            cost_pred = net.fit(x, y, burn_in=(pass_ep % re_burn < burn_in),
                                    resample_momentum=(it_count % resample_its == 0),
                                    resample_prior=(it_count % resample_prior_its == 0))
            it_count += 1
            cost_train[pass_ep] += cost_pred
            nb_samples += len(x)
        
        cost_train[pass_ep] /= nb_samples
        net.update_lr(pass_ep)

        if pass_ep % re_burn >= burn_in and pass_ep % sim_steps == 0:
            net.save_sampled_net(max_samples=N_saves)

    Pf_ref, B_ref, x_mc_norm, x_mc_scaled, _ = lstate.monte_carlo_estimate(mcs_samples)
    X_uq = torch.tensor(x_mc_norm, dtype=torch.float32)

    print('pf_ref', Pf_ref)

    y_bnn_pred = net.sample_predict(X_uq, Nsamples=N_saves)
    y_bnn_mean = torch.mean(y_bnn_pred*y_max, 0)
    y_bnn_std = torch.std(y_bnn_pred*y_max , 0)

    print('pf_surrogate ' , (((y_bnn_mean<0).sum()) / torch.tensor(mcs_samples)).item() )

    U_f = np.abs(y_bnn_mean) / y_bnn_std

    U_min_args = np.argsort(U_f.reshape(-1))     #ordering arguments from min to max
    X_new = x_mc_scaled[U_min_args[:active_points]]  #choosing a given number of MC samples from the minimum U values
    X_new_norm = X_uq[U_min_args[:active_points]]  #choosing a given number of MC samples from the minimum U values
    x_train = torch.cat( (x_train, X_new_norm))

    Y_new_norm = lstate.eval_lstate(X_new) / y_max
    Y_new_norm = torch.tensor(Y_new_norm, dtype=torch.float32).view(active_points, 1)
    y_train = torch.cat( (y_train, Y_new_norm))

print('End training')

Samples:  100


  0%|          | 0/50 [00:00<?, ?it/s]/Users/jonathan/Documents/SGHMC/Benchmark_BNNs_StructuralReliability/methods/sghmc.py:273: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/python_arg_parser.cpp:1485.)
  d_p.add_(weight_decay, p.data)
100%|██████████| 50/50 [00:00<00:00, 241.06it/s]


pf_ref 0.0002
pf_surrogate  0.02377999946475029
Samples:  105


100%|██████████| 50/50 [00:00<00:00, 283.11it/s]


pf_ref 0.00025
pf_surrogate  0.0
Samples:  110


...

pf_ref 0.00023
pf_surrogate  9.999999747378752e-06
Samples:  190


100%|██████████| 50/50 [00:00<00:00, 195.93it/s]


pf_ref 0.00019
pf_surrogate  0.0
Samples:  195


100%|██████████| 50/50 [00:00<00:00, 159.07it/s]


pf_ref 0.00025
pf_surrogate  0.0
End training
